In [ ]:
# %load_ext autoreload

In [ ]:
# %autoreload 2

#Import everything
import io
import param
import panel as pn
import pandas as pd
import csv
import os
import random
import ipywidgets as ipw
import time
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from plotly.subplots import make_subplots
from pathlib import Path

import visualization

from tornado.ioloop import IOLoop
# from setup_stuff import read_csv, createObj
import FiberClass as fc

# import plotly

pn.extension("plotly", sizing_mode = "stretch_width", loading_color = '#00aa41')

In [ ]:
#Functions that interact with our fiber obj functions

#Read fpho data
def run_init_fiberobj(event = None):
    # .value param to extract variables properly
    # value = fpho_input.value
    # obj_name = input_1.value
    # fiber_num = input_2.value
    # animal_num = input_3.value
    # exp_date = input_4.value
    # exp_time = input_5.value

    # -------------------------------Testing purposes
    fpho_input = 'FiberPhoSig2020-12-19T17_38_46.csv'
    obj_name = 'toot'
    fiber_num = 1
    animal_num = 1
    exp_date = '2020-12-19'
    exp_time = '1:20'
    
    if fpho_input:
        try:
            # string_io = io.StringIO(file_in.decode("utf8"))
            df = pd.read_csv(fpho_input)
        except FileNotFoundError:
            print("Could not find file: " + fpho_input)
            sys.exit(2)
        except PermissionError:
            print("Could not access file: " + fpho_input)
            sys.exit(3)
    
    if df.empty:
        print("Dataframe is empty")
        sys.exit(4)
    else:
        new_obj = fc.fiberObj(df, obj_name, fiber_num, animal_num, exp_date, exp_time)
        # fiber_dict.append(new_obj)
        fiber_objs[obj_name] = new_obj
        # add to multi-selector
        existing_objs = fiber_objs
        obj_selecta.options = [*existing_objs] #Updates selector with new objects
        # add to single select for behavior
        behav_selecta.options = [*existing_objs]
        norm_selecta.options = [*existing_objs]
        
    # -------------------------------Testing purposes
    
    
#     if value:
#         try:
#             #Add to list
#             input_params = []
#             input_params.extend([obj_name, fiber_num, animal_num, exp_date, exp_time])
#             string_io = io.StringIO(value.decode("utf8"))
#             df = pd.read_csv(string_io) #Read into dataframe
#         except FileNotFoundError:
#             print("Could not find file: " + fpho_input)
#             sys.exit(2)
#         except PermissionError:
#             print("Could not access file: " + fpho_input)
#             sys.exit(3)
    
#     if df.empty:
#         print("Dataframe is empty")
#         sys.exit(4)
#     else:
#         #Create new object
#         new_obj = fc.fiberObj(df, input_params[0], input_params[1], input_params[2], input_params[3], input_params[4])
#         #Add to dict
#         fiber_objs[input_params[0]] = new_obj
#         existing_objs = fiber_objs
#         obj_selecta.options = [*existing_objs] #Updates selector with new objects

# @pn.depends('obj_selecta.value', watch = True)
def run_plot_raw_trace(event = None):
    # .value param to extract variables properly
    selected_objs = obj_selecta.value
    
    #For len of selected objs, create and plot raw signal graph
    for objs in selected_objs:
        temp = fiber_objs[objs]
        plot_pane = pn.pane.Plotly(height = 300, sizing_mode = "stretch_width") #Creates pane for plotting
        plot_pane.object = temp.raw_signal_trace() #Sets figure to plot variable
        # plot_pane.trigger('object')
        plot_raw_card.append(plot_pane) #Add figure to template

def run_normalize_a_signal(event = None):
    # .value param to extract variables properly
    selected_objs = norm_selecta.value
    #For len of selected objs, create and plot raw signal graph
    for objs in selected_objs:
        temp = fiber_objs[objs]
        plot_pane = pn.pane.Plotly(height = 900, sizing_mode = "stretch_width") #Creates pane for plotting
        plot_pane.object = temp.normalize_a_signal(pick_signal.value, pick_reference.value) #Sets figure to plot variable
        # plot_pane.trigger('object')
        norm_sig_card.append(plot_pane) #Add figure to template
        
#Read behavior data
def run_import_behavior_data(event = None):
    # behav = behav_input.filename
    behav = 'beh2020-12-19t17_38_46.csv' # ------ Testing purposes
    
    selected_obj = behav_selecta.value
    obj=fiber_objs[selected_obj]
    upload_beh_card.append('# behavior added to ' + selected_obj + ' successfully')
    # fpho = obj.fpho_data_df
    
    if behav:
        obj.import_behavior_data(behav)
        print("success")
    else:
        print("Error reading behavior data")
        sys.exit(5)
        
        



In [ ]:

#Template and widget declarations
ACCENT_COLOR="#0072B5"
template = pn.template.MaterialTemplate(site = 'Donaldson Lab: Fiber Photometry', title = 'FiberPho GUI',
                                       sidebar = ["**Upload CSV** and set **Input Parameters** for your fiber object here"],
                                        main = [], accent_base_color = ACCENT_COLOR, header_background = ACCENT_COLOR
                                       )

#Dict of objects
fiber_objs = {}

# ----------------------------------------------------- # 
#Init fiberobj Widget

#Input variables
input_1 = pn.widgets.TextInput(name = 'Object Name', width = 90, placeholder = 'String')
input_2 = pn.widgets.IntInput(name = 'Fiber Number', width = 90, placeholder = 'Int')
input_3 = pn.widgets.IntInput(name = 'Animal Number', width = 90, placeholder = 'Int')
input_4 = pn.widgets.TextInput(name = 'Exp Date', width = 90, placeholder = 'Date')
input_5 = pn.widgets.TextInput(name = 'Exp Time', width = 90, placeholder = 'Time')
input_col = pn.Column(input_1, input_2, input_3, input_4, input_5)
fpho_input = pn.widgets.FileInput(name = 'Upload FiberPho Data', accept = '.csv') #File input parameter

#Buttons
upload_button = pn.widgets.Button(name = 'Create Object', button_type = 'primary', width = 500, sizing_mode = 'stretch_width', align = 'end')
upload_button.on_click(run_init_fiberobj) #Button action

#Box
init_obj_box = pn.WidgetBox('# Input Params', fpho_input, input_col, upload_button)

# ----------------------------------------------------- # 


# ----------------------------------------------------- # 
#Plot raw signal Widget

#Input vairables
obj_selecta = pn.widgets.MultiSelect(name = 'Fiber Objects', value = [], options = [], )

#Buttons
plot_raw_btn = pn.widgets.Button(name = 'Plot Raw Signal', button_type = 'primary', width = 200, sizing_mode = 'stretch_width', align = 'start')
plot_raw_btn.on_click(run_plot_raw_trace)

#Box
plot_options = pn.Column(obj_selecta, plot_raw_btn)
plot_raw_widget = pn.WidgetBox('# Options', plot_options)
plot_raw_card = pn.Card(plot_raw_widget, title = 'Plot Raw Signal', background = 'WhiteSmoke', width = 600)
# ----------------------------------------------------- # 
#Normalize signal to reference Widget
#Input vairables

norm_selecta = pn.widgets.MultiSelect(name = 'Fiber Objects', value = [], options = [], )
pick_signal = pn.widgets.Select(name = 'Signal', options = ['Green', 'Isosbestic', 'Red'])
pick_reference = pn.widgets.Select(name = 'Reference', options = ['Green', 'Isosbestic', 'Red'])
#Buttons
norm_sig_btn = pn.widgets.Button(name = 'Normalize Signal', button_type = 'primary', width = 200, sizing_mode = 'stretch_width', align = 'start')
norm_sig_btn.on_click(run_normalize_a_signal)

#Box
norm_options = pn.Column(norm_selecta, pick_signal, pick_reference, norm_sig_btn)
norm_sig_widget = pn.WidgetBox(norm_options)
norm_sig_card = pn.Card(norm_sig_widget, title = 'Normalize to a reference', background = 'WhiteSmoke', width = 600)


# ----------------------------------------------------- # 
#Add Behavior Widget

#Input variables
behav_input = pn.widgets.FileInput(name = 'Upload Behavior Data', accept = '.csv') #File input parameter
behav_selecta = pn.widgets.Select(name = 'Fiber Objects', value = [], options = [], )


#Buttons
upload_beh_btn = pn.widgets.Button(name = 'Read Behavior Data', button_type = 'primary', width = 200, sizing_mode = 'stretch_width', align = 'start')
upload_beh_btn.on_click(run_import_behavior_data) #Button action

#Box
behav_options = pn.Column(behav_selecta, behav_input, upload_beh_btn)
upload_beh_widget = pn.WidgetBox('# Behavior Analysis', behav_options)
upload_beh_card = pn.Card(upload_beh_widget, title = 'Behavior Functions', background = 'WhiteSmoke', width = 600)

# ----------------------------------------------------- # 

# ----------------------------------------------------- # 
#Plot Behaviors on raw trace widget




#Append widgets to gui template
template.sidebar.append(init_obj_box)
template.main.append(plot_raw_card)
template.main.append(norm_sig_card)
template.main.append(upload_beh_card)
# template.servable()
# template.main.append(visuals)

In [ ]:
server = pn.serve(template, start = False, show = False, websocket_max_message_size = 10485760000,
                  verbose = True,
                  threaded = False
                 )
server.start()

In [ ]:
val = behav_selecta.value
data = fiber_objs[val]
data.normalize_a_signal('Green', 'Isosbestic')


In [ ]:
#server.stop()